# Table of Contents

- [1. Project Title]()
- [2. Library import and data loading](#2-Library-import-and-data-loading)
  - [2.1 Library imports](##Library-import)
  - [2.2 Loading uncleand dataset](##Loading-uncleaned-data)
- [3. Data Analysis,Preparation and Wrangling](#3-Data-Analysis-Preparation-and-Wrangling)
  - [3.1 Handling missing rows](##Handling-missing-rows)
  - [3.2 Handling the currency symbols and comma format](##Handling-the-currency-symbols-and-comma-format)
  - [3.3 Handling non-valid column names](##Handling-non_valid-column-names)
  - [3.4 Handling missing values in columns: fuel_type](##Handling-missing-values-in-columns:-fuel_type)
  - [3.5 Handling missing values and comma format in columns: mileage](##Handling-missing-values-and-comma-format-in-columns:-mileage)
  - [3.6 Handling missing values and different value standard in columns: engine_size](##Handling-missing-values-and-different-value-standard-in-columns:-engine_size)
  - [3.7 Handling wrong data type: year](##Handling-wrong-data-type:-year)
  - [3.8 Handling unneeded columns](##Handling-unneeded-columns)
  - [3.9 Handling categorical data](##Handling-categorical-data)
  - [3.10 Saving the cleaned data into csv](##Saving-the-cleaned-data-into-csv)

# Predicting resale value of cars
## Dataset: UK’s craiglist car

### Group Members
- Shedrach Ezenwali  300377414

____

# 2-Library-import-and-data-loading

____

## Library-import

In [645]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression

## Loading-uncleaned-data

In [646]:
df = pd.read_csv("./data/unclean_data.csv")

____

# 3-Data-Analysis-Preparation-and-Wrangling

____

In [647]:
## A look of the data set
df.head()

,model,year,price,transmission,mileage,fuel type,engine size,mileage2,fuel type2,engine size2,reference
0,C Class,2020.0,"£30,495",Automatic,NaN,Diesel,2,"1,200",NaN,NaN,/ad/25017331
1,C Class,2020.0,"£29,989",Automatic,NaN,Petrol,1.5,"1,000",NaN,NaN,/ad/25043746
2,C Class,2020.0,"£37,899",Automatic,NaN,Diesel,2,500,NaN,NaN,/ad/25142894
3,C Class,2019.0,"£30,399",Automatic,NaN,Diesel,2,"5,000",NaN,NaN,/ad/24942816
4,C Class,2019.0,"£29,899",Automatic,NaN,Diesel,2,"4,500",NaN,NaN,/ad/24913660


In [648]:
## Let's get general information about the data, the shape and also the data type

def get_data_description(df)->None:
    '''Take a dataframe and prints out a consise 
    description based on the data shape'''
    rows,cols = df.shape

    print(f'We have {rows} rows and {cols} columns in the data set')

get_data_description(df)



We have 4006 rows and 11 columns in the data set


In [649]:
## Checking of the data type of each column

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4006 entries, 0 to 4005
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         3907 non-null   object 
 1   year          3904 non-null   float64
 2   price         3907 non-null   object 
 3   transmission  3907 non-null   object 
 4   mileage       3808 non-null   object 
 5   fuel type     1329 non-null   object 
 6   engine size   3842 non-null   object 
 7   mileage2      3890 non-null   object 
 8   fuel type2    3808 non-null   object 
 9   engine size2  3808 non-null   object 
 10  reference     3907 non-null   object 
dtypes: float64(1), object(10)
memory usage: 344.4+ KB


From first look, some data type are wrong. 
<div><ins>Some few points to note</ins></div>

- for example the year is a floating point number which logically year can only be an integer.
- Price appears as a string probably due to the currency symbol

Finally there are some NaN values in the data set

In [650]:
## Getting the percentage of missing values in each column

def check_cols_with_null(df):
    '''Gets  dataframe and prints out percentage of missing values in each column'''
    print(    df.shape)
    for col in df.columns:
        print(f'{col}: {format((df[col].isnull().sum()/df.shape[0])*100,".0f")}%')

check_cols_with_null(df)

(4006, 11)
model: 2%
year: 3%
price: 2%
transmission: 2%
mileage: 5%
fuel type: 67%
engine size: 4%
mileage2: 3%
fuel type2: 5%
engine size2: 5%
reference: 2%


Fuel type has the highest missing value in the data set. This NaN values will be handled appropriately as we progress

## Handling-missing-rows

In [651]:
## Drop rows that have all columns as nan
df.dropna(inplace=True,how="all")

In [652]:
## Resetting row index
df.reset_index(inplace=True,drop=True)

In [653]:
get_data_description(df)

We have 3907 rows and 11 columns in the data set


## Handling-the-currency-symbols-and-comma-format

In [654]:
## Current price format
df.price.head()

0     £30,495
1     £29,989
2     £37,899
3     £30,399
4     £29,899
Name: price, dtype: object

In [655]:
df.price = df.price.str.replace('£|,',"",regex=True)

In [656]:
## Formatted price column
df.price.head()

0     30495
1     29989
2     37899
3     30399
4     29899
Name: price, dtype: object

## Handling-non_valid-column-names

In [657]:
#columns to rename and their corresponding new names

columns_map = {
    "fuel type":"fuel_type",
    "fuel type2":"fuel_type2",
    "engine size":"engine_size",
    "engine size2":"engine_size2"
}

df.rename(columns=columns_map,inplace = True)

In [658]:
df.head(1)

,model,year,price,transmission,mileage,fuel_type,engine_size,mileage2,fuel_type2,engine_size2,reference
0,C Class,2020.0,30495,Automatic,NaN,Diesel,2,"1,200",NaN,NaN,/ad/25017331


## Handling-missing-values-in-columns:-fuel_type

In [659]:
#initially we noticed that there was a lot of missing values in the 'fuel column' see below for percentages of missing values

check_cols_with_null(df[["fuel_type","fuel_type2"]])

(3907, 2)
fuel_type: 66%
fuel_type2: 3%


About 66% of the values in the fuel_type column is missing, at this point we can decide to drop it, however before we drop the column we need to it values to fill the fuel_types column with reduce it from 3% or possibly drop it to 0%

In [660]:
df.fuel_type2.fillna(df.fuel_type,inplace=True)

In [661]:
df.fuel_type2.isna().sum()

0

In [662]:
df.drop(["fuel_type"],axis=1,inplace=True)
df.rename(columns={'fuel_type2':'fuel_type'},inplace=True)

df.head(1)

,model,year,price,transmission,mileage,engine_size,mileage2,fuel_type,engine_size2,reference
0,C Class,2020.0,30495,Automatic,NaN,2,"1,200",Diesel,NaN,/ad/25017331


## Handling-missing-values-and-comma-format-in-columns:-mileage

In [663]:
## checking percentage of missing data in millage

check_cols_with_null(df[["mileage","mileage2"]])

(3907, 2)
mileage: 3%
mileage2: 0%


In [664]:
df.mileage.fillna(df.mileage2,inplace=True)
df.mileage = df.mileage.str.replace(",","")

In [665]:
df.mileage = pd.to_numeric(df.mileage,errors='coerce')

In [666]:
df.drop(["mileage2"],axis=1,inplace=True)

## Handling-missing-values-and-different-value-standard-in-columns:-engine_size

In [667]:
## checking percentage of missing data in millage

check_cols_with_null(df[["engine_size","engine_size2"]])

(3907, 2)
engine_size: 2%
engine_size2: 3%


In [668]:
df.engine_size2.fillna(df.engine_size,inplace=True)

In [669]:
df.engine_size2 = pd.to_numeric(df['engine_size2'],errors="coerce")

In [670]:
df.engine_size2.describe()

count    3905.000000
mean      249.148513
std       706.052083
min         0.000000
25%         2.000000
50%         2.000000
75%         2.100000
max      6208.000000
Name: engine_size2, dtype: float64

In [671]:
def convert_engine_size(engine_size):
    if engine_size > 1000:
        return round(engine_size/1000,1)
    
    return round(engine_size,1)


df.engine_size2 = df.engine_size2.apply(lambda x:convert_engine_size(x))

In [672]:
df.drop(["engine_size"],axis=1,inplace=True)

df.rename(columns={'engine_size2':'engine_size'},inplace=True)

In [673]:
df.head(1)

,model,year,price,transmission,mileage,fuel_type,engine_size,reference
0,C Class,2020.0,30495,Automatic,1200.0,Diesel,2.0,/ad/25017331


## Handling-wrong-data-type:-year

In [674]:
df.dropna(inplace=True)
df.reset_index(inplace=True,drop=True)

In [675]:
df.year = df.year.astype('int64') 

## Handling-unneeded-columns

In [676]:
# Dropping the model and reference columns.

df.drop(["model","reference"],axis=1,inplace=True)

## Handling-categorical-data

In [677]:
df = df.join(pd.get_dummies(df[['transmission',"fuel_type"]],dtype=int))

In [679]:
df.drop(["transmission","fuel_type"],axis=1,inplace=True)

In [681]:
get_data_description(df)

We have 3898 rows and 12 columns in the data set


In [682]:
df.head()

,year,price,mileage,engine_size,transmission_Automatic,transmission_Manual,transmission_Other,transmission_Semi-Auto,fuel_type_Diesel,fuel_type_Hybrid,fuel_type_Other,fuel_type_Petrol
0,2020,30495,1200.0,2.0,1,0,0,0,1,0,0,0
1,2020,29989,1000.0,1.5,1,0,0,0,0,0,0,1
2,2020,37899,500.0,2.0,1,0,0,0,1,0,0,0
3,2019,30399,5000.0,2.0,1,0,0,0,1,0,0,0
4,2019,29899,4500.0,2.0,1,0,0,0,1,0,0,0


## Saving-the-cleaned-data-into-csv

In [683]:
df.to_csv("./data/cleaned_data.csv")